In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"

import torch
import torch.nn as nn
from torch_geometric.data import DataLoader
import argparse
import numpy as np
import random
import ogb
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from graph_transformer import GT
from utils import pre_process, get_n_params
import os
import datetime
from tqdm import tqdm
from tensorboardX import SummaryWriter

In [2]:
parser = argparse.ArgumentParser(description='PyTorch implementation of relative positional encodings and relation-aware self-attention for graph Transformers')
args = parser.parse_args("")

args.dataset = 'ogbg-molhiv'
args.n_classes = 1
args.lr = 2e-4
args.n_hid = 512
args.n_heads = 8
args.n_layer = 4
args.dropout = 0.2
args.num_epochs = 100
args.k_hop_neighbors = 3
args.weight_decay = 1e-2
args.bsz      = 128
args.strategies = ['ea', 'sd', 'cn']
args.summary_node = True
args.writer = SummaryWriter(log_dir=f'runs_new/{args.dataset}/k={args.k_hop_neighbors}/strats={"-".join(args.strategies)}/{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}')
args.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device:", args.device)

device: cuda


In [3]:
print("Loading data...")
print("dataset: {} ".format(args.dataset))
dataset = PygGraphPropPredDataset(name=args.dataset, pre_transform=lambda d : pre_process(d, args), root = f'dataset/{args.k_hop_neighbors}')
evaluator = Evaluator(name=args.dataset)
split_idx = dataset.get_idx_split()
edge_dim_dict = {'ea': dataset.data.edge_attr.max().int().item() + 1, \
                 'sd': dataset.data.sd_edge_attr.max().int().item() + 1, \
                 'cn': dataset.data.cn_edge_attr.max().int().item() + 1}
model = GT(args.n_hid, args.n_classes, args.n_heads, args.n_layer, edge_dim_dict, args.dropout, args.summary_node).to(args.device)

Loading data...
dataset: ogbg-molhiv 


In [4]:
from torch.optim import AdamW

def get_optimizer(model: nn.Module, learning_rate: float = 1e-4, adam_eps: float = 1e-6,
                  weight_decay: float = 0.0) -> torch.optim.Optimizer:
    no_decay = ['bias', 'LayerNorm.weight']

    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
         'weight_decay': weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_eps)
    return optimizer

In [5]:
print('Model #Params: %d' % get_n_params(model))

criterion = torch.nn.BCEWithLogitsLoss(reduction = "mean")

optimizer = get_optimizer(model, weight_decay = args.weight_decay)

train_loader = DataLoader(dataset[split_idx["train"]], batch_size=args.bsz, shuffle=True)
valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=args.bsz, shuffle=False)
test_loader  = DataLoader(dataset[split_idx["test"]],  batch_size=args.bsz, shuffle=False)

scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=args.lr, pct_start = 0.05,\
        steps_per_epoch=len(train_loader), epochs = args.num_epochs, anneal_strategy = 'linear')

Model #Params: 8549889


In [6]:
stats = []
for epoch in range(args.num_epochs):
    model.train()
    train_loss = []
    for num_iters, data in enumerate(tqdm(train_loader)):
        data.to(args.device)
        out = model(data.x, data.batch, data.edge_index, {'ea': data.edge_attr, 'cn': data.cn_edge_attr, 'sd': data.sd_edge_attr})
        loss = criterion(out, data.y.float())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        optimizer.zero_grad()
        scheduler.step()
        
        train_loss += [loss.item()]
        
    args.writer.add_scalar("LR/epoch", optimizer.param_groups[0]['lr'], epoch + 1)
    args.writer.add_scalar("Loss/train", np.average(train_loss), epoch + 1)

    model.eval()
    with torch.no_grad():
        valid_loss = []
        y_true = []
        y_scores = []
        for num_iters, data in enumerate(tqdm(valid_loader)):
            data.to(args.device)
            out = model(data.x, data.batch, data.edge_index, {'ea': data.edge_attr, 'cn': data.cn_edge_attr, 'sd': data.sd_edge_attr})

            loss = criterion(out, data.y.float())
            valid_loss += [loss.item()]

            y_true += [data.y]
            y_scores += [out]

        input_dict = {"y_true": torch.cat(y_true), "y_pred": torch.cat(y_scores)}
        valid_rocauc = evaluator.eval(input_dict)['rocauc']
        args.writer.add_scalar("Loss/valid", np.average(valid_loss), epoch + 1)
        args.writer.add_scalar("ROC/valid", valid_rocauc, epoch + 1)
        
        test_loss = []
        y_true = []
        y_scores = []
        for data in test_loader:
            data.to(args.device)
            out = model(data.x, data.batch, data.edge_index, {'ea': data.edge_attr, 'cn': data.cn_edge_attr, 'sd': data.sd_edge_attr})

            loss = criterion(out, data.y.float())
            test_loss += [loss.item()]

            y_true += [data.y]
            y_scores += [out]

        input_dict = {"y_true": torch.cat(y_true), "y_pred": torch.cat(y_scores)}
        test_rocauc = evaluator.eval(input_dict)['rocauc']
        args.writer.add_scalar("Loss/test", np.average(test_loss), epoch + 1)
        args.writer.add_scalar("ROC/test", test_rocauc, epoch + 1)
    
    print('Epoch %d: LR: %.5f, Train loss: %.3f Valid loss: %.3f  Valid ROC-AUC: %.3f Test loss: %.3f  Test ROC-AUC: %.3f' \
          % (epoch + 1, optimizer.param_groups[0]['lr'], np.average(train_loss), np.average(valid_loss), \
            valid_rocauc, np.average(test_loss), test_rocauc))
    stats += [[epoch, np.average(train_loss), np.average(valid_loss), valid_rocauc, np.average(test_loss), test_rocauc]]

args.writer.close()

  0%|          | 0/258 [00:00<?, ?it/s]

Epoch 1: LR: 0.00005, Train loss: 0.168 Valid loss: 0.095  Valid ROC-AUC: 0.500 Test loss: 0.138  Test ROC-AUC: 0.500


  0%|          | 0/258 [00:00<?, ?it/s]

Epoch 2: LR: 0.00008, Train loss: 0.160 Valid loss: 0.095  Valid ROC-AUC: 0.498 Test loss: 0.140  Test ROC-AUC: 0.498


  0%|          | 1/258 [00:00<00:50,  5.10it/s]

...

Epoch 98: LR: 0.00000, Train loss: 0.159 Valid loss: 0.097  Valid ROC-AUC: 0.498 Test loss: 0.137  Test ROC-AUC: 0.498


  0%|          | 0/258 [00:00<?, ?it/s]

Epoch 99: LR: 0.00000, Train loss: 0.159 Valid loss: 0.097  Valid ROC-AUC: 0.500 Test loss: 0.137  Test ROC-AUC: 0.500


100%|██████████| 33/33 [00:03<00:00, 10.23it/s]


Epoch 100: LR: -0.00000, Train loss: 0.164 Valid loss: 0.097  Valid ROC-AUC: 0.498 Test loss: 0.137  Test ROC-AUC: 0.498
